# GAN implementation by tensorflow

In [1]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# utility関数


In [2]:
def save_metrics(metrics, epoch=None):
    plt.figure(figsize=(10,8))
    plt.plot(metrics["dis_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join("metrics", "dloss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["gen_loss"], label="generative loss", color="r")
    plt.legend()
    plt.savefig(os.path.join("metrics", "g_loss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["gen_loss"], label="generative loss", color="r")
    plt.plot(metrics["dis_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join("metrics", "both_loss" + str(epoch) + ".png"))
    plt.close()

In [3]:
# noise[[examples, 100]]から生成した画像をplot_dim(例えば4x4)で表示
def save_imgs(images, plot_dim=(5,12), size=(12,5), epoch=None):
    examples = plot_dim[0]*plot_dim[1]

    # 表示
    fig = plt.figure(figsize=size)
    for i in range(examples):
        plt.subplot(plot_dim[0], plot_dim[1], i+1)
        img = images[i, :]
        img = img.reshape((28, 28))
        plt.tight_layout()
        plt.imshow(img, cmap="Greys_r")
        plt.axis("off")
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(os.path.join("generated_figures", str(epoch) + ".png"))
    plt.close()


# モデル

In [4]:
class Generator:
    def __init__(self):
        self.reuse = False
        self.initializer = tf.contrib.layers.xavier_initializer()
        self.X_dim = 784
        self.h_dim = 128
        self.z_dim = 100

    def __call__(self, inputs, training=False):
        with tf.variable_scope('g', reuse=self.reuse):
            # reshape from inputs
            outputs = tf.reshape(inputs, [-1, self.z_dim])
           
            with tf.variable_scope("fc1"):
                weight = tf.get_variable("W", shape=[self.z_dim, self.h_dim], initializer=self.initializer)
                bias = tf.get_variable("b", shape=[self.h_dim], initializer=self.initializer)
                fc1 = tf.matmul(outputs, weight) + bias
                fc1 = tf.nn.relu(fc1)

            with tf.variable_scope("fc2"):
                weight = tf.get_variable("W", shape=[self.h_dim, self.X_dim], initializer=self.initializer)
                bias = tf.get_variable("b", shape=[self.X_dim], initializer=self.initializer)
                fc2 = tf.matmul(fc1, weight) + bias
                fc2 = tf.sigmoid(fc2)
#                fc2 = tf.reshape(fc2, [-1, 28, 28])

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return fc2

In [5]:
class Discriminator:
    def __init__(self):
        self.reuse = False
        self.h_dim = 128
        self.z_dim = 100
        self.X_dim = 784
        self.initializer = tf.contrib.layers.xavier_initializer()
    def __call__(self, inputs, training=False, name=''):
        def leaky_relu(x, leak=0.2, name='outputs'):
            return tf.maximum(x, x * leak, name=name)
#        outputs = tf.convert_to_tensor(inputs)
        outputs = tf.reshape(inputs, [-1, self.X_dim])
         
        with tf.name_scope('d' + name), tf.variable_scope('d', reuse=self.reuse):
            with tf.variable_scope("fc1"):
                weight = tf.get_variable("W", shape=[self.X_dim, self.h_dim], initializer=self.initializer)
                bias = tf.get_variable("b", shape=[self.h_dim], initializer=self.initializer)
                fc1 = tf.matmul(outputs, weight) + bias
                fc1 = tf.nn.relu(fc1)

            with tf.variable_scope("fc2"):
                weight = tf.get_variable("W", shape=[self.h_dim, 1], initializer=self.initializer)
                bias = tf.get_variable("b", shape=[1], initializer=self.initializer)
                fc2 = tf.matmul(fc1, weight) + bias
#                fc2 = tf.sigmoid(fc2)

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')
        return fc2

In [6]:
class DCGAN:
    def __init__(self, batch_size=128):
        self.batch_size = batch_size
        self.g = Generator()
        self.d = Discriminator()
        self.z_dim = 100

    def d_loss_calc(self, g_outputs, t_outputs, gradient_penalty):
        lambda_ = 10

        d_loss_train = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=t_outputs, labels=tf.ones_like(t_outputs)))
        d_loss_gen = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=g_outputs, labels=tf.zeros_like(g_outputs)))
        loss = d_loss_train + d_loss_gen
        loss = loss + lambda_*gradient_penalty
#        loss = -tf.reduce_mean(tf.log(t_outputs) + tf.log(1. - g_outputs)) + lambda_*gradient_penalty
        return loss

    def g_loss_calc(self, g_outputs):
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=g_outputs, labels=tf.ones_like(g_outputs)))
        return loss

    def loss(self, traindata, perturbed_minibatch):
        z = tf.random_uniform([self.batch_size, self.z_dim], minval=-1.0, maxval=1.0)
        generated = self.g(z, training=True)
        g_outputs = self.d(generated, training=True, name='g')
        t_outputs = self.d(traindata, training=True, name='t')
        
        alpha = tf.random_uniform(shape=[self.batch_size,1], minval=0., maxval=1.)
        

        diff = perturbed_minibatch - traindata
        interpolates = traindata + (alpha*diff)
        gradients = tf.gradients(self.d(interpolates), [interpolates])[0]
        slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), reduction_indices=[1]))
        gradient_penalty = tf.reduce_mean((slopes-1.)**2)
        #disc_cost += lambd*gradient_penalty
        
        # d_losses

        # add each losses to collection
        tf.add_to_collection(
            'g_losses',
            self.g_loss_calc(g_outputs))

        tf.add_to_collection(
            'd_losses',
            self.d_loss_calc(g_outputs, t_outputs, gradient_penalty)
        )

        return {
            self.g: tf.add_n(tf.get_collection('g_losses'), name='total_g_loss'),
            self.d: tf.add_n(tf.get_collection('d_losses'), name='total_d_loss'),
        }

    def d_train(self, losses, learning_rate=1e-4, beta1=0.5, beta2=0.9):
        d_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1, beta2=beta2)
        d_opt_op = d_opt.minimize(losses[self.d], var_list=self.d.variables)
        return d_opt_op

    def g_train(self, losses, learning_rate=1e-4, beta1=0.5, beta2=0.9):
        g_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1, beta2=beta2)
        g_opt_op = g_opt.minimize(losses[self.g], var_list=self.g.variables)
        return g_opt_op

    def sample_images(self, row=5, col=12, inputs=None, epoch=None):
        images = self.g(inputs, training=True)
        #save_imgs(images, epoch=epoch)
        return images

In [7]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
import numpy as np

#X_train = np.load("irasutoya_face_1813x96x96x3_jpg.npy")
#X_train = X_train/255
X_dim = 784
z_dim = 100
batch_size = 128
epochs = 100000
display_epoch = 1000
param_save_epoch = 10000
loss = {"dis_loss":[], "gen_loss":[]}

In [ ]:
p_noise = tf.placeholder(tf.float32, [None, z_dim])
noise_check = np.random.uniform(-1, 1, size=[60, z_dim]).astype(np.float32)

X = tf.placeholder(tf.float32, [None, X_dim])
X_p = tf.placeholder(tf.float32, [None, X_dim])

dcgan = DCGAN()
losses = dcgan.loss(traindata=X, perturbed_minibatch=X_p)
d_train_op = dcgan.d_train(losses)
g_train_op = dcgan.g_train(losses)

saver = tf.train.Saver()
#%debug
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        for _ in range(1):
            # ノイズから生成データを生成
            #noise = np.random.uniform(-1, 1, size=[batch_size, z_dim])

            if epoch % display_epoch == 0:
                imgs_gen = dcgan.sample_images(inputs=noise_check).eval()
                print("saving images")
                save_imgs(imgs_gen, epoch=epoch)

            # 訓練データを抜粋
            #rand_index = np.random.randint(0, X_train.shape[0], size=batch_size)

            X_mb, _ = mnist.train.next_batch(batch_size)
            # x + delta vector
            X_mb_p = X_mb + 0.5*X_mb.std()*np.random.random(X_mb.shape)

            _, d_loss_value = sess.run([d_train_op, losses[dcgan.d]], feed_dict={X: X_mb, X_p:X_mb_p})

        _, g_loss_value, = sess.run([g_train_op, losses[dcgan.g]], feed_dict={X: X_mb, X_p:X_mb_p})

        # 結果をappend
        loss["dis_loss"].append(d_loss_value)
        loss["gen_loss"].append(g_loss_value)
        print("epoch:" + str(epoch))
        # グラフの描画（余裕があったら）
        if epoch % display_epoch == 0:
            save_metrics(loss, epoch)

        if epoch % param_save_epoch == 0:
            saver.save(sess, "./model/dcgan_model" + str(epoch) + ".ckpt")

saving images
epoch:0
epoch:1
epoch:2
epoch:3
epoch:4
epoch:5
epoch:6
epoch:7
epoch:8
epoch:9
epoch:10
epoch:11
epoch:12
epoch:13
epoch:14
epoch:15
epoch:16
epoch:17
epoch:18
epoch:19
epoch:20
epoch:21
epoch:22
epoch:23
epoch:24
epoch:25
epoch:26
epoch:27
epoch:28
epoch:29
epoch:30
epoch:31
epoch:32
epoch:33
epoch:34
epoch:35
epoch:36
epoch:37
epoch:38
epoch:39
epoch:40
epoch:41
epoch:42
epoch:43
epoch:44
epoch:45
epoch:46
epoch:47
epoch:48
epoch:49
epoch:50
epoch:51
epoch:52
epoch:53
epoch:54
epoch:55
epoch:56
epoch:57
epoch:58
epoch:59
epoch:60
epoch:61
epoch:62
epoch:63
epoch:64
epoch:65
epoch:66
epoch:67
epoch:68
epoch:69
epoch:70
epoch:71
epoch:72
epoch:73
epoch:74
epoch:75
epoch:76
epoch:77
epoch:78
epoch:79
epoch:80
epoch:81
epoch:82
epoch:83
epoch:84
epoch:85
epoch:86
epoch:87
epoch:88
epoch:89
epoch:90
epoch:91
epoch:92
epoch:93
epoch:94
epoch:95
epoch:96
epoch:97
epoch:98
epoch:99
epoch:100
epoch:101
epoch:102
epoch:103
epoch:104
epoch:105
epoch:106
epoch:107
epoch:108
epoch:

In [ ]:
import numpy as np

a = np.arange(10)
print(a)
print(a.shape)

b = a - 1
print(b)
print(b.shape)
print(np.abs(b))
#np.abs
#tf.pow((tf.abs(delta)-1), 2))

In [ ]:
a = [[[0],[1],[2]],
     [[0],[1],[2]],
     [[0],[1],[2]],
     [[0],[1],[2]]]

b = tf.reduce_sum(a, axis=0)

with tf.Session() as sess:
    c = sess.run(b)
    
print(np.array(a).shape)
print(c)